In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras as K
from tensorflow.keras import layers as L

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices

np.random.rand()

train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)

train['action'] = ((train['resp'].values) > 0).astype(int)

features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, train.columns.str.contains('feature')]

y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

In [ ]:
#SEED = 1111

np.random.rand()

def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = L.Input(shape=(num_columns,))
    x = L.BatchNormalization()(inp)
    x = L.Dropout(dropout_rates)(x)
    for i in range(3):
        x = L.Dense(hidden_units)(x)
        x = L.BatchNormalization()(x)
        x = L.Activation(K.activations.swish)(x)
        x = L.Dropout(dropout_rates)(x)
    
    x = L.Dense(num_labels)(x)
    out = L.Activation("sigmoid")(x)

    model = K.models.Model(inputs=inp, outputs=out)
    
    model.summary()
    
    model.compile(
        optimizer = tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss = K.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics = K.metrics.AUC(name="AUC"),
    )

    return model

epochs = 250
batch_size = 4096
hidden_units = 350
dropout_rates = 0.2
label_smoothing = 1e-2
learning_rate = 1e-3

K.backend.clear_session()

clf = create_mlp(
    len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

#clf.load_weights('./model.h5') #use starting from 2nd run

clf.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2)

In [ ]:
# save model
clf.save(f'model.h5')

# inference
th = 0.503
models = [clf]
f = np.median
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)